In [1]:
# ref: https://www.youtube.com/watch?v=Ss_GdU0KqE0&t=182s
# comparativos entre modelos que aceitam funções: https://gorilla.cs.berkeley.edu/blogs/4_open_functions.html
# https://gorilla.cs.berkeley.edu/leaderboard.html


In [1]:
# testing Ollama - https://github.com/samwit/agent_tutorials/blob/main/ollama_agents/llama3_local/testing_ollama.py

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Local Llama3 
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    max_new_tokens=512)

prompt = ChatPromptTemplate.from_template("Write me a 500 word article on {topic} from the perspective of a {profession}. ")

# using LangChain Expressive Language chain syntax
chain = prompt | llm | StrOutputParser()



# print(chain.invoke({"topic": "LLMs", "profession": "shipping magnate"}))

for chunk in chain.stream({"topic": "LLMs", "profession": "shipping magnate"}):
    print(chunk, end="", flush=True)

**"The Future of Freight: How Large Language Models are Revolutionizing Logistics"**

As a shipping magnate, I've spent my career navigating the complexities of global trade. From optimizing routes to managing supply chains, every decision has been driven by a singular goal: getting goods from point A to point B as efficiently and effectively as possible.

But in recent years, a new player has entered the scene that's changing the game for logistics professionals like myself: Large Language Models (LLMs). These AI-powered tools are capable of processing vast amounts of data, generating insights, and even making decisions – all with unprecedented speed and accuracy.

At first glance, it might seem counterintuitive to think that LLMs would have a significant impact on the shipping industry. After all, aren't they just fancy language generators? But trust me, these models are far more than that. They're game-changers.

One of the most significant advantages of LLMs is their ability to ana

In [2]:
import json
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "favorite_food": {
            "title": "Fav Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age","fav_food"],
}

llm = ChatOllama(
    model="llama3.1",
    format="json",
    temperature=0.1,
    max_new_tokens=512
    )

messages = [
    HumanMessage(
        content="Please tell me about a person using the following JSON schema:"
    ),
    HumanMessage(content="{schema}"),
    HumanMessage(
        content="Now, just considering the schema, tell me about a person named John who is 35 years old and loves pizza."
    ),
]

prompt = ChatPromptTemplate.from_messages(messages)

#converting the json schema to a string
dumps = json.dumps(json_schema, indent=2)

# chain = prompt | llm | StrOutputParser()
chain = prompt | llm | JsonOutputParser()

response = chain.invoke({"schema": dumps})
print(response)
print(type(response))

{'properties': {'name': {'type': 'string'}, 'age': {'type': 'integer'}, 'hobbies': {'type': 'array', 'items': {'type': 'string'}}}}
<class 'dict'>


In [3]:
# json schema
# format="json"


import json
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "favorite_food": {
            "title": "Favorite Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age", "favorite_food"],
}

llm = ChatOllama(
    model="llama3.1",
    format="json",
    temperature=0.1,
    max_new_tokens=512
)

messages = [
    HumanMessage(content="Please provide information about a person using the following JSON schema:"),
    HumanMessage(content="{schema}"),
    HumanMessage(content="Generate information for a person named John who is 35 years old and loves pizza."),
]

prompt = ChatPromptTemplate.from_messages(messages)

# Convert the JSON schema to a string
schema_str = json.dumps(json_schema, indent=2)

chain = prompt | llm | JsonOutputParser()

response = chain.invoke({"schema": schema_str})
print(json.dumps(response, indent=2))
print(type(response))

{
  "name": "John",
  "age": 35,
  "hobbies": [
    "pizza"
  ]
}
<class 'dict'>


In [4]:
# Ollama functions
# pydantic
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
#from langchain_experimental.llms.ollama_functions import OllamaFunctions

# Pydantic Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")

context = """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde."""

# Prompt template llama3
prompt = PromptTemplate.from_template(
    """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a smart assistant take the following context and question below and return your answer in JSON.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
QUESTION: {question} \n
CONTEXT: {context} \n
JSON:
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
 """
)

# Chain
llm = ChatOllama(model="llama3.1",format="json", temperature=0)

structured_llm = llm.with_structured_output(Person)
chain = prompt | structured_llm

response = chain.invoke({
    "question": "Who is taller?",
    "context": context
    })

print(response)

name='Alex' height=5.0 hair_color='blonde'


In [5]:
# bind tools

model = ChatOllama(model="llama3.1", temperature=0)

model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

response = model.invoke("what is the weather in Singapore?")

print(response)

content='' response_metadata={'model': 'llama3.1', 'created_at': '2024-09-05T19:34:13.256776959Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': {'location': 'Singapore', 'unit': 'fahrenheit'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 8453431741, 'load_duration': 56296185, 'prompt_eval_count': 182, 'prompt_eval_duration': 4884333000, 'eval_count': 25, 'eval_duration': 3462767000} id='run-25b07880-9a6f-4de7-9bfb-889f78c2d564-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Singapore', 'unit': 'fahrenheit'}, 'id': '596f3d8b-5806-4af0-b61d-ba69ba684d6d', 'type': 'tool_call'}] usage_metadata={'input_tokens': 182, 'output_tokens': 25, 'total_tokens': 207}


-------
https://python.langchain.com/v0.2/docs/how_to/tool_calling/

Repare que o LLM não executa uma função, ele apenas indica que uma função deve ser executada e os parâmetros que devem ser passados.

In [6]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]

In [7]:
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(model="llama3.1", temperature=0)
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"
llm_with_tools.invoke(query)

AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-05T19:34:23.765663701Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 10388939387, 'load_duration': 58526876, 'prompt_eval_count': 216, 'prompt_eval_duration': 7232744000, 'eval_count': 22, 'eval_duration': 3055357000}, id='run-014fef60-77d2-4fe8-9c94-62d861183111-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '252292cd-f66f-4452-b3ac-1708683bfc1b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 22, 'total_tokens': 238})

In [8]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': '194fb237-86ac-4185-90e8-5b474149c590',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'bb10b46f-25d5-4ada-bb82-f28600af1c67',
  'type': 'tool_call'}]

In [9]:
# Parsing
from langchain_core.output_parsers import PydanticToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]

-----
- https://python.langchain.com/v0.2/docs/how_to/tool_results_pass_to_model/
- https://python.langchain.com/v0.2/docs/how_to/custom_tools/#creating-tools-from-functions

In [10]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [11]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_callsb)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '5d04c0fc-cc90-4f9a-a924-bbbbee451a43', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 60}, 'id': 'f8057ba1-efad-4e27-a598-a62d811aea43', 'type': 'tool_call'}]


In [23]:


tools_calls = ai_msg.tool_calls

# Parse tool name and arguments
tool_name = tools_calls[0]['name']
arguments = tools_calls[0]['args']
#city = arguments['city']
print(tool_name)
print(arguments)



multiply
{'a': 3, 'b': 12}


In [12]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-05T19:35:17.032481013Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}, {'function': {'name': 'add', 'arguments': {'a': 11, 'b': 60}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 22906487983, 'load_duration': 59756470, 'prompt_eval_count': 221, 'prompt_eval_duration': 7058527000, 'eval_count': 107, 'eval_duration': 15786267000}, id='run-38cb0390-52cd-47ea-9601-fc232c9dcc7b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '5d04c0fc-cc90-4f9a-a924-bbbbee451a43', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 60}, 'id': 'f8057ba1-efad-4e27-a598-a62d811aea43', 'type': 'tool_call'}], usage_metadata={'input_tokens': 221, 'output_tokens': 107, 'total_tokens': 328}),
 ToolMessage(content='36', name='multiply', tool

In [13]:
llm_with_tools.invoke(messages)

AIMessage(content='Based on the tool call responses, I can provide the following answers:\n\n* 3 * 12 = 36\n* 11 + 49 = 60', response_metadata={'model': 'llama3.1', 'created_at': '2024-09-05T19:35:26.913373Z', 'message': {'role': 'assistant', 'content': 'Based on the tool call responses, I can provide the following answers:\n\n* 3 * 12 = 36\n* 11 + 49 = 60'}, 'done_reason': 'stop', 'done': True, 'total_duration': 9858929835, 'load_duration': 24358526, 'prompt_eval_count': 113, 'prompt_eval_duration': 4392205000, 'eval_count': 34, 'eval_duration': 5223377000}, id='run-bd1c1a72-9699-4ea5-9e26-5667f564b184-0', usage_metadata={'input_tokens': 113, 'output_tokens': 34, 'total_tokens': 147})

In [14]:
llm_with_tools.invoke(messages).content

'Based on the tool call responses, I can provide the following answers:\n\n* 3 * 12 = 36\n* 11 + 49 = 60'

**Tool calling agents, like those in LangGraph, use this basic flow to answer queries and solve tasks.**